# Validation neutralization assays versus `polyclonal` fits
Compare actual measured neutralization values for specific mutants to the `polyclonal` fits.

Import Python modules:

In [ ]:
import os
import pickle

import altair as alt

import pandas as pd

import yaml

Read configuration and validation assay measurements:

In [ ]:
with open("config.yaml") as f:
    config = yaml.safe_load(f)
    
validation_ic50s = pd.read_csv(config["validation_ic50s"], na_filter=None)

validation_ic50s

Now get the predictions by the averaged `polyclonal` model fits:

In [ ]:
validation_vs_prediction = []
for antibody, antibody_df in validation_ic50s.groupby("antibody"):
    with open(os.path.join(config["escape_dir"], f"{antibody}.pickle"), "rb") as f:
        model = pickle.load(f)
    validation_vs_prediction.append(model.icXX(antibody_df))
    
validation_vs_prediction = pd.concat(validation_vs_prediction, ignore_index=True)

validation_vs_prediction

Now plot the results.
We will plot the **median** across the replicate `polyclonal` fits to different deep mutational scanning replicates.
This is an interactive plot that you can mouse over for details:

In [ ]:
corr_chart = (
    alt.Chart(validation_vs_prediction)
    .encode(
        x=alt.X("measured IC50", scale=alt.Scale(type="log")),
        y=alt.Y(
            "median_IC50",
            title="predicted IC50 from DMS",
            scale=alt.Scale(type="log"),
        ),
        facet=alt.Facet("antibody", columns=4, title=None),
        color=alt.Color("lower_bound", title="lower_bound"),
        tooltip=[
            alt.Tooltip(c, format=".3g") if validation_vs_prediction[c].dtype == float
            else c
            for c in validation_vs_prediction.columns.tolist()
        ],
    )
    .mark_circle(filled=True, size=60, opacity=0.6)
    .configure_axis(grid=False)
    .resolve_scale(y="independent", x="independent")
    .properties(width=150, height=150)
)

corr_chart